In [4]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.restoration import estimate_sigma, denoise_nl_means
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
import os
from skimage.io import imread  # For loading images

def create_background_mask(image_gray):
    thresh = threshold_otsu(image_gray)
    binary_image = image_gray <= thresh
    cleared_image = clear_border(binary_image)
    label_image = label(cleared_image)
    regions = regionprops(label_image)

    largest_area = 0
    background_label = 0
    for region in regions:
        if region.area > largest_area:
            largest_area = region.area
            background_label = region.label

    background_mask = label_image == background_label
    return background_mask

def adjust_denoising_parameters(sigma_est):
    if sigma_est <= 15:
        return 0.6 * sigma_est, 7, 11
    elif sigma_est <= 30:
        return 0.9 * sigma_est, 5, 6
    elif sigma_est <= 50:
        return 0.2 * sigma_est, 9, 17
    else:
        return 0.4 * sigma_est, 11, 23

def denoise_and_plot(image_path):
    noisy_image_gray = imread(image_path, as_gray=True)  # Load and convert to grayscale

    sigma_est = np.mean(estimate_sigma(noisy_image_gray))
    h, patch_size, patch_distance = adjust_denoising_parameters(sigma_est)
    background_mask = create_background_mask(noisy_image_gray)
    std_deviation = np.std(noisy_image_gray[background_mask])
    denoised_image = denoise_nl_means(noisy_image_gray, h=h, patch_size=patch_size,
                                      patch_distance=patch_distance, fast_mode=True)

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(noisy_image_gray, cmap='gray')
    plt.title('Noisy Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(denoised_image, cmap='gray')
    plt.title(f'Denoised Image (Std Dev: {std_deviation:.2f})')
    plt.axis('off')

    plt.tight_layout(pad=3.0)
    plt.suptitle('Noisy Image and Denoised Image', fontsize=16)
    plt.show()

noisy_image_data= '../TraditionalMethods/PhantomData/200mT'
denoise_and_plot(noisy_image_data)

OSError: ImageIO does not generally support reading folders. Limited support may be available via specific plugins. Specify the plugin explicitly using the `plugin` kwarg, e.g. `plugin='DICOM'`